##### `raw_od_uuid 20~24`를 바탕으로 사용자의 지역별 빈도수 컬렉션 생성
shp 파일이 로컬에 있기에, DB의 파일을 로컬에서 처리 후, 병합할 예정.

1. 20년 데이터로 빈도수를 조사한 새로운 컬렉션 생성    
2. 21~24년 데이터로 각각 빈도수 구하고, 구한 빈도수를 새로운 컬렉션에 삽입    
    i. 삽입할 때 uuid와 region이 동일한 데이터는 컬렉션에 빈도수만 추가    
    ii. 없다면, 새로 레코드 생성

In [1]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pymongo import ReplaceOne

In [2]:
# 데이터베이스 연결
load_dotenv()
client = MongoClient(os.getenv('DB_ADR'),
          username=os.getenv('DB_USER'),
          password=os.getenv('DB_PASSWORD'),
          authSource=os.getenv('DB_AuthSource'),
          authMechanism=os.getenv('DB_AuthMechanism'))
db = client.get_database(os.getenv('DB_Collection'))

#컬렉션 불러오기 
#cl_raw_od_uuid_2020=db.get_collection("raw_od_uuid_2020")  #35,653,705
#cl_raw_od_uuid_2021=db.get_collection("raw_od_uuid_2021")  #33,920,055
#cl_raw_od_uuid_2022=db.get_collection("raw_od_uuid_2022")  #29,558,249
#cl_raw_od_uuid_2023=db.get_collection("raw_od_uuid_2023")  #23,412,023
#cl_raw_od_uuid_2024=db.get_collection("raw_od_uuid_2024")   #11,781,589

In [3]:
# 센서스 데이터 로드
os.chdir('../')
location_label_path=os.getcwd()+'/data/행정구역구분/BND_SIGUNGU_PG.shp'
# 행정동 데이터 좌표계 변환
location_label=gpd.read_file(location_label_path, encoding='euc-kr')
location_label.to_crs(epsg=4326, inplace=True)

# 특별시 및 광역시 전처리
metropolitan_city={'11':'서울', '21':'부산', '22':'대구', '23':'인천', '24':'광주', '25':'대전', '26':'울산'}
def update_dong(row):
    if row['SIGUNGU_CD'][:2] in metropolitan_city.keys():
        region_prefix = metropolitan_city.get(row['SIGUNGU_CD'][:2], '')
        if region_prefix:
            return f"{region_prefix} {row['SIGUNGU_NM']}"
    return row['SIGUNGU_NM']

# 센서스 데이터 정보 불러오기
census_col=['시도코드','시도명칭','시군구코드','시군구명칭','읍면동코드','읍면동명칭']
raw_census=pd.read_excel(os.getcwd()+'/data/행정구역구분/센서스_공간정보_지역_코드.xlsx')
def make_census_dict(raw_census):
    raw_census=raw_census.drop(index=0, axis=1)
    data=raw_census.values.tolist()
    census=pd.DataFrame(data, columns=census_col)
    census=census.set_index('시도코드')
    census=census['시도명칭']
    census=census.drop_duplicates()
    census_dict=census.to_dict()
    return census_dict
census_dict=make_census_dict(raw_census)

#지역 구분 칼럼 추가
def Region_col_add(row):
    if int(row['SIGUNGU_CD'][:2]) in census_dict.keys():
        region = census_dict.get(int(row['SIGUNGU_CD'][:2]), '')
        return region

In [4]:
# 사용할 geopandas 데이터프레임 전처리
location_label['SIGUNGU_NM'] = location_label.apply(update_dong, axis=1)
location_label['Region'] = location_label.apply(Region_col_add, axis=1)
location_label=location_label[['SIGUNGU_NM','Region','geometry']]
location_label

,SIGUNGU_NM,Region,geometry
0,서울 종로구,서울특별시,"POLYGON ((126.98 37.631, 126.97 37.63, 126.97 ..."
1,서울 중구,서울특별시,"POLYGON ((127.02 37.572, 127.02 37.572, 127.02..."
2,서울 용산구,서울특별시,"POLYGON ((126.97 37.555, 126.97 37.555, 126.97..."
3,서울 성동구,서울특별시,"POLYGON ((127.04 37.573, 127.04 37.573, 127.04..."
4,서울 광진구,서울특별시,"POLYGON ((127.1 37.572, 127.1 37.572, 127.1 37..."
...,...,...,...
245,함양군,경상남도,"POLYGON ((127.7 35.758, 127.7 35.758, 127.7 35..."
246,거창군,경상남도,"POLYGON ((127.88 35.906, 127.88 35.905, 127.88..."
247,합천군,경상남도,"POLYGON ((128.1 35.831, 128.1 35.83, 128.1 35...."
248,제주시,제주특별자치도,"MULTIPOLYGON (((126.17 33.283, 126.17 33.283, ..."


In [ ]:
#삽입할 컬렉션
user_coordinate_area='user_coordinate_area_'
#원본 컬렉션
raw_od_uuid='raw_od_uuid_'
years=['2024', '2023', '2022', '2021', '2020']

In [6]:
chunk_size = 100000

for year in years:
    cl_raw_od_uuid = db.get_collection(raw_od_uuid + year)
    cl_user_coordinate_area = db.get_collection(user_coordinate_area + year)
    total_inserted = 0

    cursor = cl_raw_od_uuid.find({}, {"uuid": 1, 'time_end': 1, 'destination_lng': 1, 'destination_lat': 1, "_id": 1})

    batch = []
    for doc in cursor:
        batch.append({
            "_id": doc["_id"],
            "uuid": doc["uuid"],
            "time_end": doc["time_end"],
            "destination_lat": doc["destination_lat"],
            "destination_lng": doc["destination_lng"],
            "geometry": Point(doc["destination_lng"], doc["destination_lat"])
        })

        # 일정 개수마다 처리
        if len(batch) >= chunk_size:
            gdf = gpd.GeoDataFrame(batch, geometry="geometry", crs="EPSG:4326")
            gdf = gpd.sjoin(gdf, location_label, how="left", predicate="within")
            gdf = gdf[['_id', 'uuid', 'time_end', 'destination_lat', 'destination_lng', 'Region', 'SIGUNGU_NM']]
            gdf = gdf.rename(columns={'Region': 'region', 'SIGUNGU_NM': 'destination_area'})

            operations = [ReplaceOne({"_id": row["_id"]}, row, upsert=True) for row in gdf.to_dict("records")]
            cl_user_coordinate_area.bulk_write(operations)

            total_inserted += len(gdf)
            print(f"{len(gdf)} inserted... (total so far: {total_inserted})")

            batch.clear()

    # 남은 데이터 처리
    if batch:
        gdf = gpd.GeoDataFrame(batch, geometry="geometry", crs="EPSG:4326")
        gdf = gpd.sjoin(gdf, location_label, how="left", predicate="within")
        gdf = gdf[['_id', 'uuid', 'time_end', 'destination_lat', 'destination_lng', 'Region', 'SIGUNGU_NM']]
        gdf = gdf.rename(columns={'Region': 'region', 'SIGUNGU_NM': 'destination_area'})

        operations = [ReplaceOne({"_id": row["_id"]}, row, upsert=True) for row in gdf.to_dict("records")]
        cl_user_coordinate_area.bulk_write(operations)

        total_inserted += len(gdf)

    print(f'{user_coordinate_area + year} 컬렉션에 {total_inserted} 개 데이터 삽입 완료')

100000 inserted... (total so far: 100000)
100000 inserted... (total so far: 200000)
100000 inserted... (total so far: 300000)
100000 inserted... (total so far: 400000)
100000 inserted... (total so far: 500000)
100000 inserted... (total so far: 600000)
100000 inserted... (total so far: 700000)
100000 inserted... (total so far: 800000)
100000 inserted... (total so far: 900000)
100000 inserted... (total so far: 1000000)
100000 inserted... (total so far: 1100000)
100000 inserted... (total so far: 1200000)
100000 inserted... (total so far: 1300000)
100000 inserted... (total so far: 1400000)
100000 inserted... (total so far: 1500000)
100000 inserted... (total so far: 1600000)
100000 inserted... (total so far: 1700000)
100000 inserted... (total so far: 1800000)
100000 inserted... (total so far: 1900000)
100000 inserted... (total so far: 2000000)
100000 inserted... (total so far: 2100000)
100000 inserted... (total so far: 2200000)
100000 inserted... (total so far: 2300000)
100000 inserted... (